<a href="https://colab.research.google.com/github/a0932384743/node-api/blob/master/%E2%80%9Cvideo_to_bvh_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Initial imports, install, initializations 
####Be patient it'll take some time :)

In [1]:
!git clone https://github.com/Dene33/keras_Realtime_Multi-Person_Pose_Estimation.git
!pip install configobj
!apt install ffmpeg
!apt-get install blender

!apt install python2.7 python-pip
!git clone https://github.com/Dene33/hmr.git
!wget https://people.eecs.berkeley.edu/~kanazawa/cachedir/hmr/models.tar.gz && tar -xf models.tar.gz
!mv models hmr/
!pip2 install -r hmr/requirements.txt
!mkdir hmr/output
!mkdir hmr/output/csv
!mkdir hmr/output/images
!mkdir hmr/output/csv_joined
!mkdir hmr/output/bvh_animation

import os
os.chdir('keras_Realtime_Multi-Person_Pose_Estimation')
!bash model/get_keras_model.sh
!mkdir sample_jsons
!mkdir sample_videos
!mkdir sample_images

exec(open('model_load.py').read())
!pip2 install opendr==0.77

Cloning into 'keras_Realtime_Multi-Person_Pose_Estimation'...
remote: Enumerating objects: 196, done.
remote: Total 196 (delta 0), reused 0 (delta 0), pack-reused 196
Receiving objects: 100% (196/196), 29.49 MiB | 19.89 MiB/s, done.
Resolving deltas: 100% (80/80), done.
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34546 sha256=3ac39e37e17f6f42cedaf73e6126cea292b9d0db6997c25b7230323b4927221c
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  blender-data fonts-dejavu fonts-dejavu-core fonts-dejavu-extr

Using TensorFlow backend.
W0903 03:09:52.479827 140543527556992 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0903 03:09:52.517850 140543527556992 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0903 03:09:52.532064 140543527556992 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0903 03:09:52.577027 140543527556992 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0903 03:09:54.400346 1405435

     |████████████████████████████████| 245kB 6.5MB/s 
  Created wheel for opendr: filename=opendr-0.77-cp27-cp27mu-linux_x86_64.whl size=6826929 sha256=1597566af34441882cea9b99aa1c6e8d32c6bf6927a40374e9d35c80dcc280ca
  Stored in directory: /root/.cache/pip/wheels/5c/cd/7c/8f4035d040aacef26bccc11cebfe69cc44871e020b1f5e598b
Successfully built opendr


In [0]:
rm -rf /content/sample_data

In [0]:
!mv /content/demo.py  /content/hmr/demo.py
!mv /content/models.py  /content/hmr/src/models.py
!mv /content/config.py  /content/hmr/src/config.py
!mv /content/data_utils.py  /content/hmr/src/util/data_utils.py
!mv /content/openpose.py  /content/hmr/src/util/openpose.py
!mv /content/batch_lbs.py  /content/hmr/src/tf_smpl/batch_lbs.py
!mv /content/batch_smpl.py  /content/hmr/src/tf_smpl/batch_smpl.py
!mv /content/projection.py  /content/hmr/src/tf_smpl/projection.py

In [0]:
cd /content/keras_Realtime_Multi-Person_Pose_Estimation/

/content/keras_Realtime_Multi-Person_Pose_Estimation


##Upload video

In [0]:
#upload video
exec(open('upload_videos.py').read())

##Process the video
####<font color='green'>`24`</font> in `video_to_images.sh` stands for desired fps rate at which you want to convert video to images

In [0]:
#convert to images, specify fps rate
!bash video_to_images.sh 20

#2d pose estimation. For each image creates corresponding .json file with format
#similar to output .json format of openpose (https://github.com/CMU-Perceptual-Computing-Lab/openpose)
exec(open('2d_pose_estimation.py').read())

#3d pose estimation
os.chdir('..')
!bash hmr/3dpose_estimate.sh

#convert estimated .csv files to bvh
!blender --background hmr/csv_to_bvh.blend -noaudio -P hmr/csv_to_bvh.py

##Download .bvh

In [0]:
from google.colab import files
files.download('hmr/output/bvh_animation/estimated_animation.bvh')

##Display results of 3d pose estimation (optional)

In [0]:
from IPython.display import Image, display

images = os.listdir('hmr/output/images')
extensions = {".jpg", ".png", ".gif", ".bmp", ".jpeg"}
   
   
for image in sorted(images):
  for ext in extensions:
    if image.endswith(ext):
      display(Image('hmr/output/images/'+image, width=512))

In [0]:
!tar cvf  /content/hmr/output/images.tar /content/hmr/output/images/ 

##Clear all the generated data if you want to process new video

In [0]:
!rm keras_Realtime_Multi-Person_Pose_Estimation/sample_images/*
!rm keras_Realtime_Multi-Person_Pose_Estimation/sample_jsons/*
!rm keras_Realtime_Multi-Person_Pose_Estimation/sample_videos/*
!rm hmr/output/bvh_animation/*
!rm hmr/output/csv/*
!rm hmr/output/csv_joined/*
!rm hmr/output/images/*

####Based on:


1.   **End-to-end Recovery of Human Shape and Pose** by *Angjoo Kanazawa, Michael J. Black, David W. Jacobs, Jitendra Malik *
https://github.com/akanazawa/hmr  

  Corresponding fork by *Denis Cera*: https://github.com/Dene33/hmr

2.   **Realtime Multi-Person 2D Pose Estimation using Part Affinity Fields** by *Zhe Cao, Tomas Simon, Shih-En Wei, Yaser Sheikh* \
Keras implementation by *Michal Faber*: https://github.com/michalfaber/keras_Realtime_Multi-Person_Pose_Estimation

  Corresponding fork by *Denis Cera*: https://github.com/Dene33/keras_Realtime_Multi-Person_Pose_Estimation


